## Workers Answers Data Integration
---

In [1]:
# def freq(answers):
#     freq_answ = { a:answers.count(a) for a in  set(answers)}
#     return freq_answ
# def get_aggregated_results_amazon(dfx):
#     rows =[]
#     for g,df in dfx.groupby('HITId'):
#         answers =df['Answer.category.label']
#         value_counts = answers.value_counts()
#         disease = df['Input.Disease']
#         doid = df['Input.DO_ID']
#         drug = df['Input.Drug_Brand_Name']
#         db_id = df['Input.Drug']
#         text = df['Input.context']
#         label = df['Input.Label_ID']
#         worker_id= df['WorkerId']
#         rows.append([value_counts.index[0], value_counts.values[0], len(answers), disease.values[0], doid.values[0],drug.values[0], \
#                          db_id.values[0], text.values[0], label.values[0], worker_id.values[0]  ])
            
#     res_df = pd.DataFrame(rows, columns=['relation_type','count','total_count','disease_name','do_id','drug_name','drugbank_id','text', 'label_id','worker_id'])
#     res_df['inter_agree'] = res_df.apply( lambda row: row['count']/row['total_count'], axis=1)
#     return res_df
# def get_aggregated_results_figure8(dfx):
#     rows =[]
#     for g,df in dfx.groupby('_unit_id'):
#         answers =df['choose_one_of_the_following_relations_between_the_given_drug_and_disease']
#         value_counts = answers.value_counts()
#         disease = df['disease_name']
#         doid = df['do_id']
#         drug = df['drug_name']
#         text = df['context']
#         db_id = df['drug']
#         label = df['label_id']
#         worker_id= df['_worker_id']
#         rows.append([value_counts.index[0], value_counts.values[0], len(answers), disease.values[0], doid.values[0],drug.values[0], \
#                      db_id.values[0], text.values[0], label.values[0], worker_id.values[0] ])
    
#     res_df = pd.DataFrame(rows, columns=['relation_type','count','total_count','disease_name','do_id','drug_name','drugbank_id','text', 'label_id','worker_id'])
#     res_df['inter_agree'] = res_df.apply( lambda row: row['count']/row['total_count'], axis=1)
#     return res_df

# def get_aggregated_results_figure8_2(dfx):
#     rows =[]
#     for g,df in dfx.groupby('_unit_id'):
#         answers =df['Worker Answer']
#         value_counts = answers.value_counts()
#         disease = df['disease_name']
#         doid = df['doid']
#         drug = df['drug_name']
#         text = df['context']
#         db_id = df['drug']
#         label = df['label']
#         worker_id= df['_worker_id']
#         rows.append([value_counts.index[0], value_counts.values[0], len(answers), disease.values[0], doid.values[0],drug.values[0], \
#                      db_id.values[0], text.values[0], label.values[0], worker_id.values[0] ])
    
#     res_df = pd.DataFrame(rows, columns=['relation_type','count','total_count','disease_name','do_id','drug_name','drugbank_id','text', 'label_id','worker_id'])
#     res_df['inter_agree'] = res_df.apply( lambda row: row['count']/row['total_count'], axis=1)
#     return res_df

In [3]:
import pandas as pd

In [4]:
df_fig8 =  pd.read_excel('OptimAL Crowdsourcing Microtasks - Drug Relations (May29).xlsx',sheet_name='Results_Full')
df_amz_1 =  pd.read_excel('OptimAL Crowdsourcing - Contraindications.xlsx',sheet_name='Results_500')
df_amz_2 =  pd.read_excel('OptimAL Crowdsourcing - Contraindications.xlsx',sheet_name='Results_500_1')
df_fig8_2 =  pd.read_excel('OptimALcrowdsourcing_relations_aggregated.xlsx',sheet_name='eachworker')
#df_fig8_2[['disease_name','drug_name','context','Worker Answer','_started_at']] #more than 3 workers assessing

In [6]:
def convert_crowd_df(raw):
    raw.columns = ['disease_name','drug_name','context','worker_label','time']
    top3 = raw.groupby(['disease_name','drug_name','context'])['time']\
        .nlargest(3).reset_index()
    return pd.merge(top3, raw['worker_label'].reset_index(), 
                           left_on='level_3', right_on='index', how='left').drop(columns=['level_3','index','time'])

In [7]:
crowd1 = convert_crowd_df(df_fig8[['disease_name','drug_name','context','Worker Answer','_started_at']])
crowd2 = convert_crowd_df(df_fig8_2[['disease_name','drug_name','context','Worker Answer','_started_at']])
crowd3 = convert_crowd_df(df_amz_1[['Input.annotation','Input.drugname','Input.context','Answer.category.label','WorkTimeInSeconds']])
crowd4 = convert_crowd_df(df_amz_2[['Input.annotation','Input.drugname','Input.context','Answer.category.label','WorkTimeInSeconds']])

#### Integrate

In [8]:
crowd_df = pd.DataFrame()
crowd_df = crowd_df.append(crowd1)
crowd_df = crowd_df.append(crowd2)
crowd_df = crowd_df.append(crowd3)
crowd_df = crowd_df.append(crowd4)
crowd_df = crowd_df.groupby(['disease_name','drug_name','context']).head(3)

---
## Gathering workers tables

In [10]:
labels = pd.read_csv('Expert2Relation_Mapping.csv',sep='\t')

In [12]:
crowd_df = pd.merge(crowd_df, labels, how='left', left_on ='worker_label', right_on ='text')
crowd_df = crowd_df.drop(columns=['text','worker_label']).rename(columns={'relation':'worker_label'})
crowd_df['value'] = 1

In [13]:
crowd_df['worker_label'].unique()

array(['Effect', 'IDK', 'Indication: Symptomatic Relief',
       'Indication: Treatment', 'Contraindication', nan], dtype=object)

## Create a workers consensus table

In [14]:
worker_table = pd.pivot_table(
    crowd_df,
    values='value',
    index=['disease_name','drug_name','context'],
    columns=['worker_label'],
    aggfunc='sum',
    fill_value=0
).reset_index()

In [15]:
worker_table['worker_answers'] = worker_table.sum(axis=1)
worker_table['Contraindication'] = worker_table['Contraindication']/worker_table['worker_answers']
worker_table['Effect'] = worker_table['Effect']/worker_table['worker_answers']
worker_table['IDK'] = worker_table['IDK']/worker_table['worker_answers']
worker_table['Indication: Symptomatic Relief'] = worker_table['Indication: Symptomatic Relief']/worker_table['worker_answers']
worker_table['Indication: Treatment'] = worker_table['Indication: Treatment']/worker_table['worker_answers']
worker_table['inter_agree_workers'] = worker_table.drop(columns='worker_answers').max(axis=1)

In [17]:
label_list = list(labels.relation.unique())
label_list.remove('Inconsistent'); label_list.remove('Duplicate')
consensus = []
for i in range(len(worker_table)):
    if worker_table.loc[i].inter_agree_workers > .51:
        for r in label_list:
            if worker_table.loc[i].inter_agree_workers == worker_table.loc[i][str(r)]:
                consensus.append(str(r))
    else:
        consensus.append('No consensus')

worker_table['worker_consensus'] = consensus

In [18]:
worker_table.head(3)

worker_label,disease_name,drug_name,context,Contraindication,Effect,IDK,Indication: Symptomatic Relief,Indication: Treatment,worker_answers,inter_agree_workers,worker_consensus
0,ABRUPTIO PLACENTAE,Aminocaproic Acid,INDICATIONS AND USAGE\n AMICAR is useful in en...,0.000000,0.333333,0.666667,0.0,0.0,3,0.666667,IDK
1,ABRUPTIO PLACENTAE,Aminocaproic Acid,INDICATIONS AND USAGE\nAMICAR is useful in enh...,0.000000,1.000000,0.000000,0.0,0.0,3,1.000000,Effect
2,ACHALASIA,IBANDRONIC ACID,"[None, 'Ibandronate sodium tablets are contrai...",0.666667,0.333333,0.000000,0.0,0.0,3,0.666667,Contraindication


In [20]:
worker_table.shape

(1922, 11)

In [21]:
worker_table.to_csv('worker_table.csv', index=False)